In [2]:
!pip install psycopg2-binary pandas

In [7]:
import psycopg2
import csv

# Подключение к базе данных
connection = psycopg2.connect(
    host="localhost",
    database="phonebook",
    user="postgres",
    password="1234"
)

connection.autocommit = True
cursor = connection.cursor()

# Создание таблицы PhoneBook
create_table_query = """
CREATE TABLE IF NOT EXISTS PhoneBook (
    id SERIAL PRIMARY KEY,
    username VARCHAR(255) NOT NULL UNIQUE,
    phone VARCHAR(20) NOT NULL UNIQUE
);
"""
cursor.execute(create_table_query)

# Функция проверки корректности номера телефона
def is_valid_phone(phone):
    # Проверяем, что телефон состоит только из цифр и имеет допустимую длину
    return re.match(r'^\+?\d{10,15}$', phone) is not None

# 1. Функция поиска по шаблону (часть имени или телефона)
def search_by_pattern(pattern):
    cursor.execute(
        "SELECT * FROM PhoneBook WHERE username LIKE %s OR phone LIKE %s",
        (f'%{pattern}%', f'%{pattern}%')
    )
    results = cursor.fetchall()
    if results:
        print("\nНайденные контакты:")
        for row in results:
            print(f"ID: {row[0]}, Имя: {row[1]}, Телефон: {row[2]}")
    else:
        print("Контакты не найдены.")

# 2. Функция для вставки множества пользователей с проверкой телефонов
def insert_many_users():
    users = []
    print("\nВведите данные пользователей (имя и телефон через запятую).")
    print("Для завершения ввода оставьте строку пустой.")
    
    while True:
        user_input = input("Введите имя и телефон: ").strip()
        if not user_input:
            break
        if ',' not in user_input:
            print("Ошибка: используйте формат 'имя,телефон'")
            continue
        username, phone = map(str.strip, user_input.split(',', 1))
        users.append((username, phone))
    
    incorrect_data = []
    correct_count = 0
    
    for username, phone in users:
        if not is_valid_phone(phone):
            incorrect_data.append((username, phone))
            continue
        
        try:
            cursor.execute(
                "INSERT INTO PhoneBook (username, phone) VALUES (%s, %s) ON CONFLICT DO NOTHING",
                (username, phone)
            )
            if cursor.rowcount > 0:
                correct_count += 1
        except psycopg2.Error as e:
            print(f"Ошибка при вставке {username}: {e}")
    
    print(f"\nУспешно добавлено: {correct_count} контактов")
    if incorrect_data:
        print("\nНекорректные данные (не добавлены):")
        for username, phone in incorrect_data:
            print(f"Имя: {username}, Телефон: {phone}")

# Функция для вставки данных из CSV
def insert_from_csv():
    with open('C:/Users/S/Desktop/PP2_lab10/contacts.csv', mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # пропустить заголовок
        for row in reader:
            try:
                cursor.execute(
                    "INSERT INTO PhoneBook (username, phone) VALUES (%s, %s) ON CONFLICT (username) DO NOTHING",
                    (row[0], row[1])
                )
            except psycopg2.Error as e:
                print(f"Ошибка при вставке данных {row[0]}: {e}")

# Функция для вставки данных через консоль
def insert_from_console():
    username = input("Enter username: ")
    phone = input("Enter phone: ")
    try:
        cursor.execute(
            "INSERT INTO PhoneBook (username, phone) VALUES (%s, %s) ON CONFLICT (username) DO NOTHING",
            (username, phone)
        )
    except psycopg2.Error as e:
        print(f"Ошибка при вставке данных: {e}")

# Функция для обновления данных
def update_phonebook(username, new_phone=None, new_username=None):
    if new_phone:
        cursor.execute(
            "UPDATE PhoneBook SET phone = %s WHERE username = %s",
            (new_phone, username)
        )
    if new_username:
        cursor.execute(
            "UPDATE PhoneBook SET username = %s WHERE username = %s",
            (new_username, username)
        )

# Функция для поиска данных по имени или телефону
def search_phonebook(username=None, phone=None):
    if username:
        cursor.execute(
            "SELECT * FROM PhoneBook WHERE username = %s",
            (username,)
        )
    elif phone:
        cursor.execute(
            "SELECT * FROM PhoneBook WHERE phone = %s",
            (phone,)
        )
    else:
        cursor.execute("SELECT * FROM PhoneBook")
    
    results = cursor.fetchall()
    for row in results:
        print(row)

# Функция для удаления данных по имени или телефону
def delete_from_phonebook(username=None, phone=None):
    if username:
        cursor.execute(
            "DELETE FROM PhoneBook WHERE username = %s",
            (username,)
        )
    elif phone:
        cursor.execute(
            "DELETE FROM PhoneBook WHERE phone = %s",
            (phone,)
        )

def main_menu():
    while True:
        print("\nPhoneBook Management System")
        print("1. Показать все контакты")
        print("2. Добавить контакт из CSV")
        print("3. Добавить контакт вручную")
        print("4. Обновить контакт")
        print("5. Поиск контакта")
        print("6. Поиск по шаблону")
        print("7. Добавить несколько контактов")
        print("8. Удалить контакт")
        print("9. Выход")
        
        choice = input("Выберите действие: ")
        
        if choice == '1':
            search_phonebook()
        elif choice == '2':
            insert_from_csv()
            print("Данные из CSV добавлены (без дубликатов)")
        elif choice == '3':
            insert_from_console()
        elif choice == '4':
            username = input("Введите имя для обновления: ")
            new_phone = input("Новый телефон (оставьте пустым чтобы не менять): ") or None
            new_username = input("Новое имя (оставьте пустым чтобы не менять): ") or None
            update_phonebook(username, new_phone, new_username)
        elif choice == '5':
            search_by = input("Искать по имени (1) или телефону (2)? ")
            if search_by == '1':
                username = input("Введите имя: ")
                search_phonebook(username=username)
            elif search_by == '2':
                phone = input("Введите телефон: ")
                search_phonebook(phone=phone)
        elif choice == '6':
            pattern = input("Введите часть имени или телефона для поиска: ")
            search_by_pattern(pattern)
        elif choice == '7':
            insert_many_users()
        elif choice == '8':
            delete_by = input("Удалить по имени (1) или телефону (2)? ")
            if delete_by == '1':
                username = input("Введите имя: ")
                delete_from_phonebook(username=username)
            elif delete_by == '2':
                phone = input("Введите телефон: ")
                delete_from_phonebook(phone=phone)
        elif choice == '9':
            break
        else:
            print("Неверный выбор")

# Запуск программы
if __name__ == "__main__":
    main_menu()
    cursor.close()
    connection.close()


PhoneBook Management System
1. Показать все контакты
2. Добавить контакт из CSV
3. Добавить контакт вручную
4. Обновить контакт
5. Поиск контакта
6. Поиск по шаблону
7. Добавить несколько контактов
8. Удалить контакт
9. Выход


Выберите действие:  1


(2, 'Elsa', ' 87475693232')
(3, 'Dana', ' 87472123665')
(6, 'Elsa', '87475693232')
(7, 'Dana', '87472123665')
(10, 'Elsa', '87475693232')
(11, 'Dana', '87472123665')
(14, 'Elsa', '87475693232')
(15, 'Dana', '87472123665')
(18, 'Elsa', '87475693232')
(19, 'Dana', '87472123665')
(22, 'Elsa', '87475693232')
(23, 'Dana', '87472123665')
(25, 'anar', '87856963232')
(26, 'jj', '87456963636')
(28, 'Elsa', '87475693232')
(29, 'Dana', '87472123665')
(1, 'Alua', '111111111')
(5, 'Alua', '111111111')
(9, 'Alua', '111111111')
(13, 'Alua', '111111111')
(17, 'Alua', '111111111')
(21, 'Alua', '111111111')
(27, 'Alua', '111111111')
(4, 'Issa', '88888888888')
(8, 'Issa', '88888888888')
(12, 'Issa', '88888888888')
(16, 'Issa', '88888888888')
(20, 'Issa', '88888888888')
(24, 'Issa', '88888888888')
(30, 'Issa', '88888888888')
(31, 'John', '99999999999')
(32, 'Doe', '11111111111')
(33, 'Alua', '87055879669')
(34, 'Elsa', '87475693232')
(35, 'Dana', '87472123665')
(36, 'Issa', '87071142893')

PhoneBook Manag

Выберите действие:  9


In [ ]:
# Закрытие соединения с базой данных
cursor.close()
connection.close()